In [2]:
'''

得到与作者相同的输入数据集
'''

Archive:  images.zip
   creating: images/
   creating: images/bedroom_601_730/
  inflating: images/bedroom_601_730/00000.jpg  
  inflating: images/bedroom_601_730/00000.json  
  inflating: images/bedroom_601_730/bedroom_601.json  
 extracting: images/bedroom_601_730/bedroom_601.png  
  inflating: images/bedroom_601_730/bedroom_602.json  
 extracting: images/bedroom_601_730/bedroom_602.png  
  inflating: images/bedroom_601_730/bedroom_603.json  
 extracting: images/bedroom_601_730/bedroom_603.png  
  inflating: images/bedroom_601_730/bedroom_604.json  
 extracting: images/bedroom_601_730/bedroom_604.png  
  inflating: images/bedroom_601_730/bedroom_605.json  
 extracting: images/bedroom_601_730/bedroom_605.png  
  inflating: images/bedroom_601_730/bedroom_606.json  
 extracting: images/bedroom_601_730/bedroom_606.png  
  inflating: images/bedroom_601_730/bedroom_607.json  
 extracting: images/bedroom_601_730/bedroom_607.png  
  inflating: images/bedroom_601_730/bedroom_608.json  
 extra

In [2]:
# # 得到标签
# ## 得到标签列表
# '''
# 所有的标签列表，实际上存在数据的只有52类
# '''

# import json  
# # 假设你的JSON文件名为'data.json'  
# json_file_path = '/home/shanxinyuan/common_disk/code/PTM/jiabu/0224_跑批/tools/img_test/json_files_202406/jiabu_skus_valid_0607.json'  
  
# # 使用'with'语句来打开文件并自动关闭它  
# with open(json_file_path, 'r', encoding='utf-8') as file:  
#     # 使用json.load()函数从文件中读取JSON数据  
#     data = json.load(file)  
# labels = []  
  
# # 遍历shapes列表中的每个字典，并提取label字段  
# for shape in data['shapes']:  
#     labels.append(shape['label'])  
# print(labels)
# print(len(labels))





FileNotFoundError: [Errno 2] No such file or directory: '/home/shanxinyuan/common_disk/code/PTM/jiabu/0224_跑批/tools/img_test/json_files_202406/jiabu_skus_valid_0607.json'

In [6]:
def deal_with(filename,label):
    if label[-5]  in '0123456789':
        # print(filename,label,":","nums")
        label =  label[:-5] + label[-4:]
    label2 = label.replace("（","(")
    label2 = label2.replace("）",")")
    if label2 != label :
        pass
        # print(filename,label,":","()")
    return label2
def get_labels(labels,folder_path):
    for filename in os.listdir(folder_path):
        if filename == "00000.json":
            print(filename)
            continue 
        json_file_path = os.path.join(folder_path, filename)


        if os.path.isfile(json_file_path) and '.json' in json_file_path:
#             print(filename)

            with open(json_file_path, 'r', encoding='utf-8') as file:  
                data = json.load(file)
            for shape in data['shapes']:
                label = shape['label']
                label = deal_with(filename,label)
                if label not in labels :
                    
                    labels.append(label)
                    # print(label)
#                 else:
#                     print(1)
    return labels 



def add_data(folder_path,data_list_train,data_list_val,data_list_test,labels):
    now_labels = []
    random.seed(42)
    ans =0 
    seeds_list = [random.random() for i in range(len(os.listdir(folder_path)))]
    for filename in os.listdir(folder_path):
        if filename == "00000.json":
            print(filename)
            continue 
        json_file_path = os.path.join(folder_path, filename)


        if os.path.isfile(json_file_path) and '.json' in json_file_path:
            print(filename)

            with open(json_file_path, 'r', encoding='utf-8') as file:  
                data = json.load(file)
            h = data["imageHeight"]
            w = data["imageWidth"]
            sample = {}
            all_xy = []
            label_list = []
            canvas_size = [h,w]
            for shape in data['shapes']:
                # print(shape)
                xy = []
                label = shape['label']
                label = deal_with(filename,label)
                label_int = labels.index(label)+1  # label 还是 label + 1 呢？ 
 
                label_list.append(label_int)  #  label_list
                points1 = shape['points'][0]
                points2 = shape['points'][1]
                min_x = min(points1[0],points2[0])/h
                max_x = max(points1[0],points2[0])/h
                min_y = min(points1[1],points2[1])/w
                max_y = max(points1[1],points2[1])/w
                xy.append(min_x)
                xy.append(min_y)
                xy.append(max_x-min_x)
                xy.append(max_y-min_y)
                all_xy.append(xy)  # all_xy
            sample['name'] = filename
            sample['labels'] = torch.tensor(label_list)
            sample['canvas_size'] = canvas_size
            sample['filtered'] = False 
            sample['bboxes'] = torch.tensor(all_xy)
#             break
            if len(data['shapes']) == 0 :
                continue
            flag = 0 
            for shape in data['shapes']:
                label = shape['label']
                print(label)
                if label not in now_labels:
                    flag =1 
                    now_labels.append(label)
            if flag == 1 :
                data_list_train.append(sample)
            else :
                random_float = seeds_list[ans]
                ans+=1
                # random_float = random.random() 
                if random_float > 0.85:
                    data_list_test.append(sample)
                elif random_float > 0.70:
                    data_list_val.append(sample)
                else:
                    data_list_train.append(sample)
    print("train,val,test 数据集的长度：")
    print(len(data_list_train),len(data_list_val),len(data_list_test))
    return data_list_train,data_list_val,data_list_test
    


                

In [7]:
import os
import torch 
import torch.nn  as nn 
import json 
import random
'''
得到labels:
'''

labels = []
folder_path = r"/home/ubuntu/ygq/data/mixed_images"   
labels  = get_labels(labels,folder_path)
print(labels)
'''
处理数据：
'''

data_list_test = []
data_list_train = []
data_list_val = []
data_list_train,data_list_val,data_list_test = add_data(folder_path,data_list_train,data_list_val,data_list_test,labels)
print(data_list_test[:3])
# print(data_list_test)


00000.json
['床(正视)', '壁挂空调(正视)', '台灯(正视)', '落地床头柜(正视)', '床(左视)', '落地床头柜(左视)', '横向装饰画(左视)', '扫地机器人(正视)', '落地灯(右视)', '横向装饰画(正视)', '装饰画(左视)', '床与床头柜组合(正视)', '装饰画(正视)', '吊灯(正视)', '竖向装饰画(正视)', '衣帽架(正视)', '吸顶灯(正视)', '落地梳妆台(正视)', '床(右视)', '落地床头柜(右视)', '四门衣柜(左视)', '装饰画(右视)', '空气净化器(正视)', '床尾凳(正视)', '横向装饰画(右视)', '落地灯(左视)', '吸尘器(正视)', '加湿器(正视)', '多门衣柜(右视)', '单人床尾凳(正视)', '落地空调(正视)', '落地梳妆台组合(正视)', '竖向装饰画(右视)', '四门衣柜(右视)', '壁挂空调(左视)', '落地灯(正视)', '空气净化器(右视)', '桌面加湿器(正视)', '多门衣柜(左视)', '落地无镜梳妆台组合(正视)', '扫地机器人(右视)', '双门衣柜(正视)', '竖向装饰画(左视)', '台灯(左视)', '壁挂空调(右视)', '吊灯(右视)', '洗地机(正视)', '落地梳妆台组合(右视)', '四门衣柜(正视)', '吸尘器(左视)', '落地梳妆台(右视)', '吸顶灯(左视)']
bedroom_708.json
床(正视)
壁挂空调(正视)
台灯(正视)
落地床头柜(正视)
1695285451064-anchorImage.json
床(左视)
落地床头柜(左视)
横向装饰画(左视)
扫地机器人2(正视)
138_4.json
床(正视)
落地床头柜(正视)
落地灯2(右视)
横向装饰画2(正视)
17_5.json
床(左视)
落地灯(右视)
落地床头柜(左视)
装饰画(左视)
bedroom_609.json
床与床头柜组合(正视)
装饰画(正视)
扫地机器人2(正视)
1698055592901-anchorImage.json
床(正视)
壁挂空调(正视)
落地床头柜(正视)
横向装饰画(正视)
bedroom_659.json
床(正视)
吊灯(正视)
竖向装饰画(正视)
1698

/gpudisk2/liuluohang/anaconda3/envs/laydi/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
print(data_list_test[:2])

[{'name': 'bedroom_726.json', 'labels': tensor([ 1,  4, 13]), 'canvas_size': [800, 800], 'filtered': False, 'bboxes': tensor([[0.1937, 0.4850, 0.5663, 0.3475],
        [0.1300, 0.5814, 0.1462, 0.1688],
        [0.3040, 0.2903, 0.1648, 0.1673]])}, {'name': '76_4.json', 'labels': tensor([ 5,  6, 23]), 'canvas_size': [512, 512], 'filtered': False, 'bboxes': tensor([[0.1427, 0.5135, 0.6824, 0.3256],
        [0.0086, 0.6161, 0.1552, 0.1489],
        [0.1722, 0.7724, 0.1172, 0.2038]])}]


In [5]:
print(labels)
print(len(labels))

['床(正视)', '壁挂空调(正视)', '台灯(正视)', '落地床头柜(正视)', '床(左视)', '落地床头柜(左视)', '横向装饰画(左视)', '扫地机器人(正视)', '落地灯(右视)', '横向装饰画(正视)', '装饰画(左视)', '床与床头柜组合(正视)', '装饰画(正视)', '吊灯(正视)', '竖向装饰画(正视)', '衣帽架(正视)', '吸顶灯(正视)', '落地梳妆台(正视)', '床(右视)', '落地床头柜(右视)', '四门衣柜(左视)', '装饰画(右视)', '空气净化器(正视)', '床尾凳(正视)', '横向装饰画(右视)', '落地灯(左视)', '吸尘器(正视)', '加湿器(正视)', '多门衣柜(右视)', '单人床尾凳(正视)', '落地空调(正视)', '落地梳妆台组合(正视)', '竖向装饰画(右视)', '四门衣柜(右视)', '壁挂空调(左视)', '落地灯(正视)', '空气净化器(右视)', '桌面加湿器(正视)', '多门衣柜(左视)', '落地无镜梳妆台组合(正视)', '扫地机器人(右视)', '双门衣柜(正视)', '竖向装饰画(左视)', '台灯(左视)', '壁挂空调(右视)', '吊灯(右视)', '洗地机(正视)', '落地梳妆台组合(右视)', '四门衣柜(正视)', '吸尘器(左视)', '落地梳妆台(右视)', '吸顶灯(左视)']
52


In [170]:
print(len(en_labels))

52


In [26]:
ROOT_path = r'/home/ubuntu/ygq/LayOutDiffusion/data/raw_datasets/rico/pre_processed_20_25/'
torch.save(data_list_test, f'{ROOT_path}/test.pt')
torch.save(data_list_test, 'data_test.pt')

torch.save(data_list_train, 'data_train.pt')
torch.save(data_list_train, f'{ROOT_path}/train.pt')

torch.save(data_list_val, f'{ROOT_path}/val.pt')
torch.save(data_list_val, 'data_val.pt')
import torch
import io
import random

with open('data_train.pt', 'rb') as f:
    buffer = io.BytesIO(f.read())
    file=torch.load(buffer)

# print(len(file))  ### 3279 
# print(file[0])
# print(file[:10])         


In [3]:
import io
import torch
with open('data_train.pt', 'rb') as f:
    buffer = io.BytesIO(f.read())
    file=torch.load(buffer)
print(file[0])

{'name': 'bedroom_708.json', 'labels': tensor([1, 2, 3, 4]), 'canvas_size': [800, 800], 'filtered': False, 'bboxes': tensor([[0.1515, 0.4286, 0.6861, 0.4251],
        [0.1361, 0.1131, 0.2987, 0.1040],
        [0.7781, 0.5064, 0.0774, 0.1374],
        [0.7478, 0.6229, 0.1603, 0.1659]])}


/gpudisk2/liuluohang/anaconda3/envs/const_layout4/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
### 提取训练集中所有的照片名字：
train_list = []
val_list = []
test_list = []
def get_image_list(path):
    train_list = []
    with open(path, 'rb') as f:
        buffer = io.BytesIO(f.read())
        file=torch.load(buffer)
    for img in file:
        image_name = img['name']
        train_list.append(image_name)
    return train_list

train_list = get_image_list('data_train.pt')
test_list = get_image_list('data_test.pt')
val_list = get_image_list('data_val.pt')
print(len(train_list),len(val_list),len(test_list))
print(train_list)
print(val_list)
print(test_list)


270 36 50
['bedroom_708.json', '1695285451064-anchorImage.json', '138_4.json', '17_5.json', 'bedroom_609.json', '1698055592901-anchorImage.json', 'bedroom_659.json', '1698491278507-anchorImage2.json', '0240dc62f76c625c252f1588ad841550.json', '2_2.json', '1697184239032-anchorImage.json', '1698491898496-anchorImage.json', '1698067965523-anchorImage.json', 'bedroom_690.json', '1698069037129-anchorImage.json', '5b456058b8aa722acf7f54e3ee49c812.json', '07a6d2718c838b7cbf3aee419dfc8509.json', 'bedroom_675.json', '1695022246016-anchorImage.json', 'bedroom_657.json', '18_0.json', '1698057218495-anchorImage.json', '1698065873147-anchorImage.json', '69735d59747e61f38f5adcb541852338.json', 'bedroom1103_01.json', 'bedroom_664.json', '99_2.json', '3f373b6b7f304a50fc0398f9cf47fe30.json', 'bedroom_610.json', '1695781014439-anchorImage.json', '77cbe8393b05774455b46578e34a9658.json', '42_4.json', 'bedroom_711.json', '1698648893249-anchorImage.json', 'ded676ee221b22dba950028569ecf809.json', '36_2_2.json

In [3]:
import json

# 假设你的 JSON 文件名为 data.json
file_path = '/home/ubuntu/ygq/LayOutDiffusion/results/generation_outputs/rico_v9/type/rico_v9.ema_0.9999_215000.pt.samples_-1.0_elem1.json'

# 打开并读取 JSON 文件
# with open(file_path, 'r', encoding='utf-8') as file:
#     datas = json.load(file)


datas = []
with open(file_path, 'r', encoding='utf-8') as file:
    for i,line in enumerate(file):
        # 解析每一行的 JSON 列表
        item = json.loads(line)
        datas.append(item)
# 打印读取到的数据
for i,data in enumerate(datas) :
    print(i,data)


0 ['START 床(正视) 21 56 106 104 | 落地床头柜(正视) 98 73 115 92 | 装饰画(正视) 43 26 65 47 END PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD']
1 ['START 床(左视) 23 46 125 98 | 空气净化器(正视) 14 70 28 96 | 落地床头柜(左视) 5 63 24 85 END PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD']
2 ['START 吊灯(正视) 48

In [28]:

    labels =['床(正视)', '壁挂空调(正视)', '台灯(正视)', '落地床头柜(正视)', '床(左视)', '落地床头柜(左视)', '横向装饰画(左视)', '扫地机器人(正视)', '落地灯(右视)', '横向装饰画(正视)', '装饰画(左视)', '床与床头柜组合(正视)', '装饰画(正视)', '吊灯(正视)', '竖向装饰画(正视)', '衣帽架(正视)', '吸顶灯(正视)', '落地梳妆台(正视)', '床(右视)', '落地床头柜(右视)', '四门衣柜(左视)', '装饰画(右视)', '空气净化器(正视)', '床尾凳(正视)', '横向装饰画(右视)', '落地灯(左视)', '吸尘器(正视)', '加湿器(正视)', '多门衣柜(右视)', '单人床尾凳(正视)', '落地空调(正视)', '落地梳妆台组合(正视)', '竖向装饰画(右视)', '四门衣柜(右视)', '壁挂空调(左视)', '落地灯(正视)', '空气净化器(右视)', '桌面加湿器(正视)', '多门衣柜(左视)', '落地无镜梳妆台组合(正视)', '扫地机器人(右视)', '双门衣柜(正视)', '竖向装饰画(左视)', '台灯(左视)', '壁挂空调(右视)', '吊灯(右视)', '洗地机(正视)', '落地梳妆台组合(右视)', '四门衣柜(正视)', '吸尘器(左视)', '落地梳妆台(右视)', '吸顶灯(左视)']
    def get_data():
        train_list = ['bedroom_708.json', '1695285451064-anchorImage.json', '138_4.json', '17_5.json', 'bedroom_609.json', '1698055592901-anchorImage.json', 'bedroom_659.json', '1698491278507-anchorImage2.json', '0240dc62f76c625c252f1588ad841550.json', '2_2.json', '1697184239032-anchorImage.json', '1698491898496-anchorImage.json', '1698067965523-anchorImage.json', 'bedroom_690.json', '1698069037129-anchorImage.json', '5b456058b8aa722acf7f54e3ee49c812.json', '07a6d2718c838b7cbf3aee419dfc8509.json', 'bedroom_675.json', '1695022246016-anchorImage.json', 'bedroom_657.json', '18_0.json', '1698057218495-anchorImage.json', '1698065873147-anchorImage.json', '69735d59747e61f38f5adcb541852338.json', 'bedroom1103_01.json', 'bedroom_664.json', '99_2.json', '3f373b6b7f304a50fc0398f9cf47fe30.json', 'bedroom_610.json', '1695781014439-anchorImage.json', '77cbe8393b05774455b46578e34a9658.json', '42_4.json', 'bedroom_711.json', '1698648893249-anchorImage.json', 'ded676ee221b22dba950028569ecf809.json', '36_2_2.json', 'bedroom_614.json', '1698489417668-anchorImage.json', 'bedroom_618.json', 'bedroom_649.json', '14_6.json', 'bedroom_646.json', 'de976ccb96c8521c3b244f08281ffa09.json', '1698057493474-anchorImage.json', '19dcf4bae2c947d09f4827894da55978.json', '1698112380298-anchorImage.json', 'e93ed4e886d4d8662f9642b0d1fe2d28.json', 'bedroom_605.json', '1684909976302-anchorImage.json', '1695804182474-anchorImage.json', '8_5.json', '70_5.json', 'bedroom_716.json', 'bedroom_613.json', '96_7.json', '1684908720152-anchorImage.json', '1698070070824-anchorImage.json', '79_5.json', '1684912465208-anchorImage.json', 'bedroom_625.json', 'baed22a89313f45b68addb977fd4e41c.json', '144_5.json', '1699939478713-anchorImage.json', '1700015448585-anchorImage.json', 'bedroom_651.json', 'bedroom_616.json', '1694772258694-anchorImage.json', '111_6.json', '1698489099529-anchorImage.json', 'bedroom_604.json', 'bedroom_704.json', '70_7.json', 'bedroom07_352_3.json', '1698061651066-anchorImage.json', 'bedroom_718.json', '99_6.json', 'bedroom1030_03_02.json', 'bedroom_615.json', 'bedroom_631.json', 'bedroom_692.json', 'bedroom_698.json', 'bedroom_666.json', 'bedroom_702.json', '5_1.json', 'bedroom06_382_3.json', '107_5.json', '09fa86bf1f85eca53d9b264b0896a103.json', '65_6.json', 'bedroom_662.json', 'bedroom_661.json', '89_6.json', '0c748f6a7ae4a3372119174f2631d34c.json', 'bedroom_700.json', '53_0.json', 'bedroom_729.json', 'bedroom_628.json', 'bedroom_681.json', '111_5.json', '1698056387903-anchorImage.json', '1697797683305-anchorImage.json', '1695022589971-anchorImage.json', 'bedroom_712.json', 'bedroom_682.json', '1698647135530-anchorImage.json', '1684907166205-anchorImage.json', '138_5.json', 'bedroom_608.json', '60_0.json', '7_0.json', 'bedroom_720.json', 'bedroom_669.json', 'bedroom13_413_0.json', '142_7.json', 'bedroom_730.json', '88_3_2.json', 'bedroom_663.json', '1694574088799-anchorImage.json', '1684910788680-anchorImage.json', '1698063257271-anchorImage.json', 'bedroom_656.json', 'bedroom_667.json', '1695803976174-anchorImage.json', '95eade143849bfa493c160fbd88c3cc5.json', '1698066358253-anchorImage.json', '1684750469412-anchorImage.json', '731fa02985d0bb60ed78c7684d7548af.json', '122_1.json', '44_0.json', 'bedroom_611.json', '1695804395817-anchorImage.json', 'bedroom_713.json', '1696941354303-anchorImage.json', '1695028921711-anchorImage.json', 'bedroom_699.json', 'bedroom_612.json', 'bedroom12_421_3.json', '1697092488097-anchorImage.json', 'bedroom_695.json', 'bedroom1030_05.json', '5ade88e6035c1814656baab99ed190e7.json', 'bedroom_660.json', '81_5.json', 'bedroom_620.json', 'bedroom_630.json', '68323f7a30715a0a4d943f66eea635fd.json', 'bedroom_687.json', 'bedroom_647.json', 'bedroom_694.json', '1a1579a5845cafce5fbce703e73b1a29.json', 'bedroom_723.json', '14_2.json', '1_3_1.json', '314f40d7726041044799ca097245e6d7.json', '23_6.json', '94_3.json', 'bedroom_715.json', 'ccbf03d680cff551c4af08af8a722d28.json', 'bedroom_678.json', 'bedroom_640.json', '1698491278507-anchorImage.json', '1695182320119-anchorImage.json', '151f9aa147b09ac201373272233143db.json', 'bedroom_725.json', '1698112132883-anchorImage.json', 'bedroom_643.json', '1698065305127-anchorImage.json', '1695017005279-anchorImage.json', 'bedroom_707.json', '128_0.json', 'bedroom_717.json', '0_0.json', 'bedroom_727.json', '609c44f9d9a6accbf7be863ec1c61bb3.json', '0_2.json', '61_5.json', 'bedroom1109_03_15_5.json', '1697248439424-anchorImage.json', '112_3.json', 'bedroom_629.json', '70_2.json', 'bedroom_683.json', 'bedroom_623.json', 'bedroom1109_07.json', '1698069772147-anchorImage.json', 'bedroom_624.json', 'bedroom_606.json', 'bedroom_719.json', '1698647485670-anchorImage.json', '1698056743917-anchorImage.json', 'bedroom_709.json', 'bedroom09_03.json', '54_1.json', '76_2.json', '6a69c109f47a89749eb3e25e30083962.json', '88_3.json', '1_3.json', '1_4.json', 'bedroom_619.json', '1695025868148-anchorImage.json', '11_6.json', '88_6.json', '115_3.json', 'f67f90877ac92490596a5411c5560fe22.json', '1684909048609-anchorImage.json', '1700126837401-anchorImage.json', 'b7c8c7cdb44b263bde250f82d31102ad.json', '1699853357334-anchorImage.json', '1695015699109-anchorImage.json', '1697852082807-anchorImage.json', '1c7ced505ae73885d2b502de58d9572b.json', '1684898330336-anchorImage.json', 'bedroom_621.json', 'bedroom_670.json', 'bedroom04_357_3.json', 'bedroom_721.json', '1684907921487-anchorImage.json', '1700017093928-anchorImage.json', '1698062127378-anchorImage.json', '1698068344550-anchorImage.json', '56_0.json', '1698068540925-anchorImage.json', '1695803996430-anchorImage.json', 'bedroom_686.json', 'bedroom_674.json', 'bedroom_671.json', 'bedroom_722.json', '1684912189945-anchorImage.json', '54_5.json', 'f67f90877ac92490596a5411c5560fe2.json', '1697249313652-anchorImage.json', 'bedroom_684.json', 'bedroom_639.json', '1684838093034-anchorImage.json', 'bedroom_602.json', '7_6.json', '1684844826727-anchorImage.json', 'bedroom_636.json', '1684897762560-anchorImage.json', 'bedroom_601.json', '1695195080939-anchorImage.json', '1700019488386-anchorImage.json', '1698645044074-anchorImage.json', '5344b0cd76b64b7205adf4e06a7f5e4f.json', 'bedroom_672.json', '70_3.json', 'bedroom_677.json', 'bedroom_607.json', '54_4.json', '29_0.json', '1698646392220-anchorImage.json', '1694832319075-anchorImage.json', 'bedroom_703.json', '008d5135cf197aa1c2dae38803103784.json', 'bedroom_635.json', '51_2.json', '763829d105d5d566068fc84fe30b5bfa.json', 'bedroom_654.json', '88_4.json', '72_2.json', 'ebec14f2ca594521d2facf8597ece27d.json', '1698057849363-anchorImage.json', 'bedroom_691.json', '1698066134938-anchorImage.json', '98_7.json', '1698064331706-anchorImage.json', '1695105466460-anchorImage.json', '41_4.json', '26b2f0756d64aad2c91a35031cf5a858.json', '1698056623962-anchorImage.json', 'bedroom_658.json']
        val_list = ['bedroom_668.json', '1698069491374-anchorImage.json', '107_2.json', 'aa9277734e20b2647cf3fcacfd718ec2.json', '1697798012006-anchorImage.json', '37_6.json', '1694832574761-anchorImage.json', 'bedroom_637.json', 'bedroom1030_01.json', '1694836165730-anchorImage.json', '1698064489135-anchorImage.json', '93_5.json', 'bedroom_665.json', '118_6.json', '1698068156843-anchorImage.json', '1_6.json', '8cea9dc04f6879d001f825419065adc5.json', '54_3.json', '98_1.json', '52_4.json', '1698056204536-anchorImage.json', 'ac800bfa94257189ff681b4582255485.json', '83_1.json', 'bedroom_633.json', '99_3.json', 'bedroom_705.json', '36_2.json', '115_0.json', '1698069237742-anchorImage.json', '72f7e03c7c6c63a8adfc7496d629ddf7.json', 'bedroom_650.json', '61_5_2.json', '1697248140480-anchorImage.json', '1695289760176-anchorImage.json', '487c49bb16433e2dd7e90523ec0f366c.json', '1698046635254-anchorImage.json']
        test_list = ['bedroom_726.json', '76_4.json', '1698047076550-anchorImage.json', '42_3.json', '1698113315999-anchorImage.json', '112_1.json', '14_3.json', 'bedroom_710.json', '46db21046fcd41e5ecdc0e7b8ad6d848.json', '1699950851852-anchorImage.json', '11_4.json', 'b3696928331db77200d46790e8ee5c47.json', '1698320556983-anchorImage.json', 'bedroom_685.json', 'bedroom_641.json', '20b1e4715b5c80f3719961c4f6334bbe.json', 'bedroom_701.json', '1698061651066-anchorImage2.json', '138_3.json', '1695026128917-anchorImage.json', '105_3.json', '57_0.json', '117_0.json', 'bedroom_617.json', '1698112380298-anchorImage2.json', '1699941806495-anchorImage.json', '1698111132167-anchorImage.json', '1697797891280-anchorImage.json', '65_4.json', '12_1.json', '1698117125512-anchorImage.json', '48_1.json', '54b753027d81a5c088bffb0524646a26.json', '100_1.json', '42_6.json', 'bedroom_679.json', 'c01db94051b1b46ff852adf01ea30ae8.json', 'bedroom_724.json', 'bedroom_652.json', '1698056090996-anchorImage.json', 'bedroom_603.json', 'bedroom_648.json', 'bedroom1030_02_02.json', '57_3.json', '1695024940663-anchorImage.json', '0_3.json', '2db72dbf6fbb4ac387d4c070522867c0.json', 'bedroom_622.json', '1698111763133-anchorImage.json', '596d5c9abb68302c14c035e191a36553.json']
        return train_list,val_list,test_list

In [33]:
import torch 
def process():
    # data_list = []
    # ann_dir = Path(self.raw_dir) / 'layoutdata' / 'annotations'
    # for xml_path in sorted(ann_dir.glob('*.xml')):
    #     with xml_path.open() as f:
    #         root = ET.parse(f).getroot()

    #     W = float(root.find('size/width').text)
    #     H = float(root.find('size/height').text)
    #     name = root.find('filename').text

    #     elements = root.findall('layout/element')

    boxes = []
    labels_ = []
    train_list , val_list,test_list = get_data()
    all_list = train_list + val_list + test_list
    ROOT_path = r"/home/ubuntu/ygq/data/mixed_images"
    print(len(all_list))
    for json_file in all_list:
        file_path = os.path.join(ROOT_path,json_file)
        with open(file_path,'r') as f:
            elements = json.load(f)
        
        W = elements['imageHeight']
        H = elements['imageWidth']
        name =elements['imagePath']
        for element in elements['shapes']:
            # bbox
            px = element['points'][0]
            py = element['points'][1]
            # px = list(map(float, element.get('polygon_x').split()))
            # py = list(map(float, element.get('polygon_y').split()))
            x1, x2 = min(px), max(px)
            y1, y2 = min(py), max(py)
            xc = (x1 + x2) / 2.
            yc = (y1 + y2) / 2.
            width = x2 - x1
            height = y2 - y1
            b = [xc / W, yc / H,
                    width / W, height / H]
            boxes.append(b)

            # label
            l = element['label']
            labels_.append(labels.index(l))

        boxes = torch.tensor(boxes, dtype=torch.float)
        labels_ = torch.tensor(labels_, dtype=torch.long)

        # data = Data(x=boxes, y=labels_)
        # data.attr = {
        #     'name': name,
        #     'width': W,
        #     'height': H,
        #     'has_canvas_element': False,
        # }
        # data_list.append(data)   ### 所有的数据

    # shuffle with seed
    generator = torch.Generator().manual_seed(0)
    indices = torch.randperm(len(data_list), generator=generator)
    data_list = [data_list[i] for i in indices]


    N = len(data_list)
    s = [270,270+36]

In [36]:
import os
import json
process()

NameError: name 'torch_geometric' is not defined

In [35]:
?

In [144]:
# !ls

In [69]:
# # common_disk/ygq/code/LayoutGeneration/LayoutDiffusion/eval_src/net/fid_rico.pth.tar
# import torch 
# checkpoint = torch.load('/home/shanxinyuan/common_disk/ygq/code/LayoutGeneration/LayoutDiffusion/eval_src/net/fid_rico.pth.tar')
# print(checkpoint)

OrderedDict([('pos_token', tensor([[[ 0.8094,  1.0914,  0.9286,  ..., -0.2261,  0.0482,  0.3774]],

        [[ 0.2805,  0.8743,  0.1105,  ..., -0.1804,  0.9011,  1.1042]],

        [[ 0.1131,  0.4626,  0.8629,  ..., -0.0728,  0.7017,  1.0734]],

        ...,

        [[ 0.8869,  0.4961,  0.6969,  ...,  1.1229,  0.4705,  0.7949]],

        [[ 1.1185,  0.1332,  0.6493,  ...,  0.6062, -0.0366,  0.6671]],

        [[ 0.7224,  0.4726,  0.2567,  ...,  0.3850,  0.4851,  0.3983]]],
       device='cuda:0')), ('emb_label.weight', tensor([[-0.7179,  1.7526,  0.0437,  ...,  0.1241, -1.2520,  1.1328],
        [ 1.4982,  0.6142, -1.0919,  ...,  1.0481,  1.5320, -1.1842],
        [ 0.5350, -0.7237, -0.5622,  ...,  1.0000,  2.2670, -1.1342],
        ...,
        [-0.0533,  0.1672, -0.1301,  ..., -1.2056, -0.2522, -0.0840],
        [-1.0013, -0.5031, -1.8840,  ..., -1.6912,  1.6424, -0.3774],
        [-0.3611, -0.3430, -1.2845,  ...,  0.7319, -2.7237, -0.2994]],
       device='cuda:0')), ('fc_bbox.weig

In [96]:
# for name, param in checkpoint.items():
#     print(f"Layer: {name} | Size: {param.size()}")

[{'name': 'bedroom_647.json',
  'labels': tensor([0, 3, 1]),
  'canvas_size': [800, 800],
  'filtered': False,
  'bboxes': tensor([[0.0747, 0.5710, 0.7362, 0.3553],
          [0.5043, 0.2972, 0.1764, 0.2590],
          [0.7346, 0.6377, 0.1356, 0.1261]])},
 {'name': 'bedroom_694.json',
  'labels': tensor([ 0,  1, 19,  3]),
  'canvas_size': [800, 800],
  'filtered': False,
  'bboxes': tensor([[0.1393, 0.5201, 0.6864, 0.4492],
          [0.7715, 0.6206, 0.1316, 0.1347],
          [0.3929, 0.0259, 0.1682, 0.1105],
          [0.6336, 0.3051, 0.1328, 0.2034]])},
 {'name': '107_5.json',
  'labels': tensor([ 0,  1, 17, 19,  3]),
  'canvas_size': [512, 512],
  'filtered': False,
  'bboxes': tensor([[0.1992, 0.5254, 0.6133, 0.2773],
          [0.1362, 0.5794, 0.1219, 0.1333],
          [0.3820, 0.6921, 0.2425, 0.1438],
          [0.4131, 0.0308, 0.1972, 0.0870],
          [0.1870, 0.3247, 0.0855, 0.1911]])},
 {'name': '138_5.json',
  'labels': tensor([ 0,  1, 19, 11,  2]),
  'canvas_size': [512,

In [41]:
import json

# # 原始 JSON 数据
# with open('/home/shanxinyuan/common_disk/ygq/code/LayoutGeneration/LayoutDiffusion/results/generation_outputs/rico_2/ungen/rico_2.ema_0.9999_035000.pt.samples_-1.0_elem1.json', 'r', encoding='utf-8') as file:
#     json_data = json.load(file)
# 打开并读取 JSON 文件
with open('/home/ubuntu/ygq/LayOutDiffusion/results/generation_outputs/rico_v7/type/rico_v7.ema_0.9999_000600.pt.samples_-1.0_elem1.json', 'r', encoding='utf-8') as file:
    lines = file.readlines()

# with open('/home/shanxinyuan/common_disk/ygq/code/LayoutGeneration/LayoutDiffusion/results/generation_outputs/rico_1/ungen/rico_1.ema_0.9999_000000.pt.samples_-1.0_elem1.json', 'r', encoding='utf-8') as file:
#     lines = file.readlines()
# # 初始化一个列表来存储解析后的数据
parsed_data = []

# 处理每一行
for line in lines:
    # 去除行末的换行符
    line = line.strip()
    
    # 将 JSON 数据转换为字符串
    json_str = json.dumps(json.loads(line))
    
    # 使用 unicode_escape 编码将 Unicode 转义字符转换为中文字符
    decoded_str = json_str.encode().decode('unicode_escape')
    
    # 将字符串转换回 JSON 数据并添加到结果列表
    parsed_data.append(json.loads(decoded_str))

# 打印结果
for data in parsed_data:
    print(data)

['29 124 121 88 85 115 6 71 扫地机器人(右视) 壁挂空调(左视) 119 51 28 111 27 55 62 START 横向装饰画(左视) 92 57 73 92 57 28 53 79 35 93 107 86 115 84 20 51 16 5 桌面加湿器(正视) 74 61 57 落地梳妆台组合(右视) 床尾凳(正视) 45 97 6 82 73 落地梳妆台组合(右视) 多门衣柜(左视) 落地灯(右视) 68 | 空气净化器(右视) 123 27 95 PAD 56 20 59 101 94 四门衣柜(左视) 18 77 79 88 59 END 25 48 70 四门衣柜(左视) PAD 43 91 57 15 52 93 48 36 装饰画(右视) 落地梳妆台组合(右视) 36 75 31 桌面加湿器(正视) 吸顶灯(左视) 83 78 42 52 86 PAD 壁挂空调(正视) 60 78 112 30 空气净化器(右视) 102 127 100 77 21 17 122 69 55 116 61 27 85 46 42 11 102 118 39']
['23 51 吸尘器(正视) 多门衣柜(右视) 落地床头柜(正视) 58 46 7 125 98 台灯(正视) 40 45 落地梳妆台组合(右视) 四门衣柜(左视) 53 落地灯(左视) 14 78 吸尘器(正视) | PAD 120 114 47 床(左视) 24 118 落地梳妆台(正视) 51 26 床(左视) 126 100 台灯(正视) 25 92 40 57 46 桌面加湿器(正视) 24 109 19 25 单人床尾凳(正视) 竖向装饰画(正视) 床(右视) 38 吊灯(正视) 88 38 66 空气净化器(正视) 落地灯(右视) 单人床尾凳(正视) 62 98 71 22 126 7 33 落地床头柜(左视) 99 53 84 落地无镜梳妆台组合(正视) 68 26 25 24 95 61 102 空气净化器(正视) 34 120 20 单人床尾凳(正视) 119 118 94 96 0 4 33 58 58 112 29 台灯(左视) 52 85 23 74 102 99 116 18 34 40 68 64 122 75 33 2 19 25 105 

In [39]:
import torch
import io
import random
import pickle
# with open('/home/shanxinyuan/common_disk/ygq/code/LaAyoutGeneration/LayoutDiffusion/results/generation_outputs/rico_2/ungen/processed.pt', 'rb') as f:
#     buffer = io.BytesIO(f.read())
#     file=torch.load(buffer)


with open('/home/shanxinyuan/common_disk/ygq/code/LayoutGeneration/LayoutDiffusion/results/generation_outputs/rico_2/tye/processed.pt', 'rb') as f:
    layouts_final = pickle.load(f)
print(layouts_final)
# # 初始化一个列表来存储解析后的数据
# parsed_data = []

# # 处理每一个 JSON 数组
# for data in layouts_final:
#     # 将 JSON 数据转换为字符串
#     json_str = json.dumps(data)
    
#     # 使用 unicode_escape 编码将 Unicode 转义字符转换为中文字符
#     decoded_str = json_str.encode().decode('unicode_escape')
    
#     # 将字符串转换回 JSON 数据并添加到结果列表
#     parsed_data.append(json.loads(decoded_str))

# # 打印结果
# for data in parsed_data:
#     print(data)


UnpicklingError: invalid load key, '['.

In [106]:


#     gold_bboxes = torch.zeros(len(gt), 20, 4)
#     gold_labels = torch.zeros(len(gt), 20).long()
#     gold_padding_mask = torch.zeros(len(gt), 20).bool()
#     for i, layout in enumerate(gt):
#         gold_bboxes[i][0 : len(layout["labels"])] = convert_ltwh_to_ltrb(
#             layout["bboxes"]
#         )
#         gold_labels[i][0 : len(layout["labels"])] = layout["labels"]
#         gold_padding_mask[i][0 : len(layout["labels"])] = 1

[{'name': 'bedroom_647.json', 'labels': tensor([0, 3, 1]), 'canvas_size': [800, 800], 'filtered': False, 'bboxes': tensor([[0.0747, 0.5710, 0.7362, 0.3553],
        [0.5043, 0.2972, 0.1764, 0.2590],
        [0.7346, 0.6377, 0.1356, 0.1261]])}, {'name': 'bedroom_694.json', 'labels': tensor([ 0,  1, 19,  3]), 'canvas_size': [800, 800], 'filtered': False, 'bboxes': tensor([[0.1393, 0.5201, 0.6864, 0.4492],
        [0.7715, 0.6206, 0.1316, 0.1347],
        [0.3929, 0.0259, 0.1682, 0.1105],
        [0.6336, 0.3051, 0.1328, 0.2034]])}, {'name': '107_5.json', 'labels': tensor([ 0,  1, 17, 19,  3]), 'canvas_size': [512, 512], 'filtered': False, 'bboxes': tensor([[0.1992, 0.5254, 0.6133, 0.2773],
        [0.1362, 0.5794, 0.1219, 0.1333],
        [0.3820, 0.6921, 0.2425, 0.1438],
        [0.4131, 0.0308, 0.1972, 0.0870],
        [0.1870, 0.3247, 0.0855, 0.1911]])}, {'name': '138_5.json', 'labels': tensor([ 0,  1, 19, 11,  2]), 'canvas_size': [512, 512], 'filtered': False, 'bboxes': tensor([[0.14

In [ ]:

labels = ['床(正视)', '落地床头柜(正视)', '装饰画(正视)', '竖向装饰画(正视)', '扫地机器人(正视)', '横向装饰画(正视)', '落地梳妆台组合(正视)', '吊灯(正视)', '落地梳妆台(正视)', '台灯(正视)', '床与床头柜组合(正视)', '落地灯(左视)', '床(左视)', '落地灯(右视)', '落地床头柜(左视)', '竖向装饰画(左视)', '落地灯(正视)', '单人床尾凳(正视)', '壁挂空调(正视)', '吸顶灯(正视)', '床尾凳(正视)', '落地无镜梳妆台组合(正视)', '床(右视)', '装饰画(右视)', '落地床头柜(右视)', '落地空调(正视)', '空气净化器(正视)', '桌面加湿器(正视)', '衣帽架(正视)', '双门衣柜(正视)', '空气净化器(右视)', '多门衣柜(左视)', '装饰画(左视)', '多门衣柜(右视)', '横向装饰画(左视)', '横向装饰画(右视)', '台灯(左视)', '吸顶灯(左视)', '加湿器(正视)', '吊灯(右视)', '四门衣柜(右视)', '四门衣柜(左视)', '壁挂空调(左视)', '四门衣柜(正视)', '壁挂空调(右视)', '落地梳妆台组合(右视)', '吸尘器(左视)', '吸尘器(正视)', '扫地机器人(右视)', '竖向装饰画(右视)', '洗地机(正视)', '落地梳妆台(右视)']
en_labels = ['Bed (Front View)', 'Chandelier (Front View)', 'Horizontal Decorative Painting (Front View)', 'Floor-standing Bedside Table (Front View)', 'Table Lamp (Front View)', 'Decorative Painting (Front View)', 'Ceiling Light (Front View)', 'Wall-mounted Air Conditioner (Front View)', 'Bed and Bedside Table Combo (Front View)', 'Bed (Left View)', 'Floor Lamp (Front View)', 'Floor-standing Bedside Table (Left View)', 'Decorative Painting (Left View)', 'Table Lamp (Left View)', 'Vertical Decorative Painting (Front View)', 'Robot Vacuum Cleaner (Front View)', 'Floor-standing Air Conditioner (Front View)', 'Single Bed Bench (Front View)', 'Bed (Right View)', 'Floor-standing Bedside Table (Right View)', 'Decorative Painting (Right View)', 'Air Purifier (Right View)', 'Floor Lamp (Right View)', 'Vertical Decorative Painting (Left View)', 'Multi-door Wardrobe (Left View)', 'Coat Rack (Front View)', 'Floor Lamp (Left View)', 'Air Purifier (Front View)', 'Multi-door Wardrobe (Right View)', 'Horizontal Decorative Painting (Left View)', 'Four-door Wardrobe (Left View)', 'Wall-mounted Air Conditioner (Left View)', 'Four-door Wardrobe (Front View)', 'Humidifier (Front View)', 'Wall-mounted Air Conditioner (Right View)', 'Floor-standing Vanity Combo (Right View)', 'Horizontal Decorative Painting (Right View)', 'Floor-standing Vanity (Front View)', 'Floor-standing Vanity Combo (Front View)', 'Vacuum Cleaner (Left View)', 'Vacuum Cleaner (Front View)', 'Robot Vacuum Cleaner (Right View)', 'Vertical Decorative Painting (Right View)', 'Floor Washing Machine (Front View)', 'Bed Bench (Front View)', 'Floor-standing Vanity (Right View)', 'Four-door Wardrobe (Right View)', 'Desktop Humidifier (Front View)', 'Ceiling Light (Left View)', 'Chandelier (Right View)', 'Floor-standing Vanity without Mirror Combo (Front View)', 'Two-door Wardrobe (Front View)']
labels = en_labels
for i in range(len(labels)):
    print(i,labels[i])

In [18]:
import json
import io
import os
from PIL import Image, ImageDraw
import torch 
import torch.nn as nn 
import matplotlib.pyplot as plt

# with open('data_train.pt', 'rb') as f:
#     buffer = io.BytesIO(f.read())
#     file=torch.load(buffer)

# print(len(file))  ### 3279 
# print(file[0])
# print(file[1])         


with open("/home/shanxinyuan/common_disk/ygq/code/LayoutGeneration/LayoutDiffusion/data/raw_datasets/rico/pre_processed_20_25/train.pt", "rb") as f: ###-
    buffer = io.BytesIO(f.read())
    datas = torch.load(buffer)
print(len(datas))  ### 3279 
print(datas[0])
print(datas[1]) 
# 示例数据
# data = {
#     'name': 'bedroom_647.json',
#     'labels': [0, 3, 1],
#     'canvas_size': [800, 800],
#     'filtered': False,
#     'bboxes': [
#         [0.0747, 0.5710, 0.7362, 0.3553],
#         [0.5043, 0.2972, 0.1764, 0.2590],
#         [0.7346, 0.6377, 0.1356, 0.1261]
#     ]
# }
for i,data in enumerate(datas): 
    if i == 20:
        break 
    print(data)
    # 读取图片
    output_dir = r"/home/shanxinyuan/common_disk/ygq/code/LayoutGeneration/LayoutDiffusion/data/draw"

    json_path = os.path.join("/home/shanxinyuan/common_disk/ygq/data/mixed_images",data['name']) # 请替换为实际图片路径
    with open(json_path, 'r', encoding='utf-8') as f:
        image_data = json.load(f)
    image_path = os.path.join("/home/shanxinyuan/common_disk/ygq/data/mixed_images",image_data['imagePath'])
    print(image_path)
    image = Image.open(image_path)
    draw = ImageDraw.Draw(image)
    canvas_size = data['canvas_size']
    print(data['labels'])
#     break 

    # 绘制边框
    for j,bbox in enumerate(data['bboxes']):
        x, y, w, h = bbox
        x1 = x * canvas_size[0]
        y1 = y * canvas_size[1]
        x2 = x1 + w * canvas_size[0]
        y2 = y1 + h * canvas_size[1]
        draw.rectangle([x1, y1, x2, y2], outline="red", width=2)
#         print(labels[int(data['labels'][i])])
        draw.text((x1, y1), str(str(labels[int(data['labels'][j])-1])), fill='red')

    # 保存结果图像
    #output_dir = "path/to/output/directory"  # 请替换为实际输出文件夹路径
    image_name = image_data['imagePath'].split(".")[0]
    os.makedirs(output_dir, exist_ok=True)
    output_path = os.path.join(output_dir, f'{i:04d}_{image_name}.png')
    image.save(output_path)

    print(f"Image saved to {output_path}")


356
{'name': '0_0.json', 'labels': tensor([1, 2, 3]), 'canvas_size': [512, 512], 'filtered': False, 'bboxes': tensor([[0.1695, 0.5043, 0.6750, 0.3456],
        [0.4334, 0.0111, 0.1782, 0.1580],
        [0.3765, 0.3056, 0.3012, 0.0866]])}
{'name': '0_2.json', 'labels': tensor([1, 4, 3]), 'canvas_size': [512, 512], 'filtered': False, 'bboxes': tensor([[0.1394, 0.4603, 0.7287, 0.4153],
        [0.0437, 0.5843, 0.1686, 0.1722],
        [0.3767, 0.2314, 0.2605, 0.1053]])}
{'name': '0_0.json', 'labels': tensor([1, 2, 3]), 'canvas_size': [512, 512], 'filtered': False, 'bboxes': tensor([[0.1695, 0.5043, 0.6750, 0.3456],
        [0.4334, 0.0111, 0.1782, 0.1580],
        [0.3765, 0.3056, 0.3012, 0.0866]])}
/home/shanxinyuan/common_disk/ygq/data/mixed_images/0_0.png
tensor([1, 2, 3])
Image saved to /home/shanxinyuan/common_disk/ygq/code/LayoutGeneration/LayoutDiffusion/data/draw/0000_0_0.png
{'name': '0_2.json', 'labels': tensor([1, 4, 3]), 'canvas_size': [512, 512], 'filtered': False, 'bboxes': 

{'id': '1', 'skuId': '10039122876681', 'cid1': '9855', 'cid1Name': '家装建材', 'cid2': '9856', 'cid2Name': '灯饰照明', 'cid3': '9904', 'cid3Name': '吸顶灯', 'transImage': 'https://img10.360buyimg.com/fit/jfs/t1/115292/30/40972/17987/650c23e5Fefa46278/1fc0f7c7b2dc71b1.png', 'horizontal': '右', 'vertical': '平视', 'combination': '', 'shape': '吸顶灯', 'style': '现代简约', 'sizeTag': '', 'tag1': '吸顶灯', 'tag2': '2', 'tag3': '', 'tag4': '', 'createUserId': '992', 'status': 1, 'deleteFlag': 0, 'createdTime': '2024-03-19 15:02:20', 'modifiedTime': '2024-05-07 16:02:45'}


39


In [21]:
config_dict = {item[1]: item[0] for item in config_labels}
print(len(config_dict))

39


In [22]:
print(config_dict)

{'单人沙发椅': '单人沙发', '双人沙发': '双人沙发', '三人及多人沙发': '三人沙发', '左转角L型沙发': '左L型沙发', '右转角L型沙发': '右L型沙发', '长方形或椭圆形茶几': '茶几', '长方形或椭圆形茶几（矮）': '无腿茶几', '圆形或方形茶几': '方形茶几', '圆形或方形茶几（矮）': '方形无腿茶几', '异形组合茶几': '异形组合茶几', '边几（矮）': '边几', '边几（高）': '立体边几', '正方形或圆形挂画（小）': '装饰画', '正方形或圆形挂画（大）': '装饰画', '竖向长方形挂画（小）': '竖向装饰画', '竖向长方形挂画': '竖向装饰画', '横向长方形挂画（大）': '横向装饰画', '横向长方形挂画（小）': '横向装饰画', '常规落地灯': '落地灯', '吸顶灯': '吸顶灯', '吊灯': '吊灯', '落地饮水机': '茶吧机', '带基站扫地机器人': '带壳扫地机器人', '不带基站扫地机器人-平视': '扫地机器人', '常规洗地机': '洗地机', '手持无线吸尘器': '吸尘器', '落地加湿器': '空气净化器', '壁挂空调': '壁挂空调', '圆形立式空调-家用': '落地空调', '方形立式空调-家用': '方形落地空调', '壁挂电视': '电视', '座式电视': '支架电视', '一字型电视柜': '电视柜', '左L型电视柜': '左L型电视柜', '右L型电视柜': '右L型电视柜', '一字型电视柜（矮）': '无腿电视柜', '一字型电视柜+茶几': '茶几电视柜组合', '左L型电视柜+茶几': '茶几左L电视柜组合', '右L型电视柜+茶几': '茶几右L电视柜组合'}


In [23]:
print(len(config_dict))

39


horizontal 
57475
57475


In [39]:
# [skuId,shape]

# 右-左视
# 中-正视

# {skuId,'shape'} # shape
# {skuId,'transImage'}
# {skuId,'horizontal'}
# "单人沙发": {'skuId'，'transImage'，horizontal'}

# [
#         "单人沙发",
#         "单人沙发椅"
#     ],
#     [
#         "双人沙发",
#         "双人沙发"
#     ],

# 得到标签
## 得到标签列表

import json  
def save_to_dirs(json_file_path,json_shape):
    with open(json_file_path, 'w', encoding='utf-8') as json_file:
        json.dump(json_shape, json_file, ensure_ascii=False, indent=4)
# 假设你的JSON文件名为'data.json'  
json_file_path = '/home/shanxinyuan/common_disk/code/PTM/jiabu/0224_跑批/tools/img_test/json_files_202406/jiabu_skus_valid_0607.json'  
hor={"右":"左视","左":"右视","中":"正视","右边":"左视","左边":"右视","中间":"正视"}
# 使用'with'语句来打开文件并自动关闭它  
json_shape ={}
json_transImage ={}
json_horizontal = {}
json_style = {}
with open(json_file_path, 'r', encoding='utf-8') as file:  
    # 使用json.load()函数从文件中读取JSON数据  
    datas = json.load(file)  
# print(datas[0])
for data in datas :
    if 'shape' in data.keys() and  data['shape'] in config_dict.keys():
        try:
            json_shape[data['skuId'] ] = config_dict[data['shape']]
        except:
            print("shape",data['shape'])
        try:
            json_transImage[data['skuId'] ] = data['transImage']
        except:
            print("transImage",data['transImage'])     
        try:
            json_horizontal[data['skuId'] ] = hor[data['horizontal']]
        except:
            json_horizontal[data['skuId'] ] = ''
#             print(data)
#             print(data['skuId'],data['horizontal'])
            print("horizontal",data['horizontal'])
        try:
            json_style[data['skuId'] ] = data['style']
        except:
             json_style[data['skuId'] ] = ''
            
#             print(data)
#             print(data['skuId'],data['style'])
#             print("style",data['style'])
print(len(json_shape))
print(len(json_horizontal))
print(len(json_shape))
# save_to_dirs(f"/home/shanxinyuan/common_disk/ygq/data/jiabu/sku_and_shape_0611.json",json_shape)
# save_to_dirs(f"/home/shanxinyuan/common_disk/ygq/data/jiabu/sku_and_transImage_0611.json",json_transImage)
# save_to_dirs(f"/home/shanxinyuan/common_disk/ygq/data/jiabu/sku_and_horizontal_0611.json",json_horizontal) 
# save_to_dirs(f"/home/shanxinyuan/common_disk/ygq/data/jiabu/sku_and_style_0611.json",json_style) 
print(len(json_style))

horizontal 
55523
55523
55523
55523


In [199]:
with open("/home/shanxinyuan/common_disk/ygq/data/jiabu/imgs_list.txt", "w") as f_w:
    with open("/home/shanxinyuan/common_disk/ygq/data/jiabu/configs_transImage.json") as f:
        load_dict = json.load(f)
        for k in load_dict:
            f_w.write(load_dict[k] + " " + k+".png" + "\n")

In [197]:
!pwd

/home/shanxinyuan/common_disk/ygq/code/LayoutGeneration_0606/LayoutDiffusion/util_scripts
